# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [17]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @carreta_vitor

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [ ]:
#Produto escolhido:
produto = 'Fanta'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [158]:
data = pd.ExcelFile('Fanta.xlsx')
df1 = data.parse('Treinamento')
df2 = data.parse('Teste')

#Limpando dataset de treinamento 
df1 = df1.replace(',','',regex=True)
df1 = df1.replace('@','',regex=True)
df1 = df1.replace(';','',regex=True)
df1 = df1.replace('  ',' ',regex=True)
df1 = df1.replace(':','',regex=True)
df1 = df1.replace('"','',regex=True)
df1 = df1.replace('\\*','',regex=True)
df1 = df1.replace('_','',regex=True)
df1 = df1.replace('\$','',regex=True)
df1 = df1.replace('rt','',regex=True)
df1 = df1.replace('//','',regex=True)
df1 = df1.replace('/','',regex=True)
df1 = df1.replace('https','',regex=True)
df1 = df1.replace('#','',regex=True)
df1 = df1.replace('\(','',regex=True)
df1 = df1.replace('\)','',regex=True)
df1 = df1.replace('\'','',regex=True)
df1 = df1.replace('\/','',regex=True)
df1 = df1.replace('\-','',regex=True)
df1 = df1.replace('!','',regex=True)
df1 = df1.replace('\n','',regex=True)
df1 = df1.replace('\*','',regex=True)

#Limpando dataset de teste
df2 = df2.replace(',','',regex=True)
df2 = df2.replace('@','',regex=True)
df2 = df2.replace(';','',regex=True)
df2 = df2.replace('  ','',regex=True)
df2 = df2.replace(':','',regex=True)
df2 = df2.replace('"','',regex=True)
df2 = df2.replace('\\*','',regex=True)
df2 = df2.replace('_','',regex=True)
df2 = df2.replace('\$','',regex=True)
df2 = df2.replace('rt','',regex=True)
df2 = df2.replace('//','',regex=True)
df2 = df2.replace('/','',regex=True)
df2 = df2.replace('https','',regex=True)
df2 = df2.replace('#','',regex=True)
df2 = df2.replace('\(','',regex=True)
df2 = df2.replace('\)','',regex=True)
df2 = df2.replace('\'','',regex=True)
df2 = df2.replace('\/','',regex=True)
df2 = df2.replace('\-','',regex=True)
df2 = df2.replace('!','',regex=True)
df2 = df2.replace('\n','',regex=True)
df2 = df2.replace('\*','',regex=True)
df2


,Teste,Relevância
0,sapacete fé fé fé só no passinho do sapacete ...,I
1,sapacete fé fé fé só no passinho do sapacete ...,I
2,amanhã no jogo do framengo vai ser só fanta ga...,I
3,sapacete fé fé fé só no passinho do sapacete ...,I
4,fantabr pensa na felicidade da galera qnd des...,I
5,sapacete fé fé fé só no passinho do sapacete ...,I
6,minha mãe fez bolo de chocolate estou amassand...,R
7,um fato guaraná simba é muito melhor que fanta...,R
8,rmcfstr atmgii essa coca fanta te lembra algu...,I
9,sapacete fé fé fé só no passinho do sapacete ...,I


In [151]:
list_relev = []
list_nrelev = []
list_all = []

#criando uma lista de palavras de acordo com a Relevância
for i in range(len(df1["Treinamento"])):
    if df1["Relevância"][i]=="R":
        list_relev+=(list(set(df1["Treinamento"][i].split())))
        list_all+=(list(set(df1["Treinamento"][i].split())))
    else:
        list_nrelev+=(list(set(df1["Treinamento"][i].split())))
        list_all+=(list(set(df1["Treinamento"][i].split())))
        
#list_relev = list(set(list_relev))
#list_nrelev = list(set(list_nrelev))
list_all = list(set(list_all))



___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [209]:
#Associando a ocorrência de uma palavra com sua relevância 
probab_relev=[]
list_relevante=[1]
probab_nrelev=[]
list_nrelevante=[1]

for i in range(len(df2.Teste)): 
    probab_relev.append([]) 
    probab_nrelev.append([]) 
    for j in range(len(df2.Teste[i].split())): 
        occur = list_relev.count(df2.Teste[i].split()[j]) 
        occur2 = list_nrelev.count(df2.Teste[i].split()[j])
        
        rel = (occur+1)/(len(list_relev)+len(list_all))
        nrel = (occur2+1)/(len(list_nrelev)+len(list_all))
                           
        probab_relev[i].append(rel)
        probab_nrelev[i].append(nrel)
                           
        relevanteI1=(list_relevante[i]*probab_relev[i][j]) 
        nrelevanteI1=(list_nrelevante[i]*probab_nrelev[i][j])
        
    list_relevante.append(relevanteI1)
    list_nrelevante.append(nrelevanteI1)

list_relevante.remove(1)
list_nrelevante.remove(1)

false_posit=0
verd_posit=0
false_negat=0
verd_negat=0

muito_relevante=0 
relevante=0
neutro=0
irrelevante=0
muito_irrelevante=0


prob_relev=[]
prob_nrelev=[]

for n in range(len(df2.Relevância)):
    if list_relevante[n] < list_nrelevante[n]:
        
        prob_relev.append(list_relevante[i])
        if df2.Relevância[i]=="R":
            
            verd_posit+=1
        else:
            false_posit+=1        
            
            
    else:
        prob_nrelev.append(list_nrelevante[i])
        if df2.Relevância[i]=="I":            
            verd_negat+=1
            
        else:            
            false_negat+=1
prob_relev.sort()
prob_nrelev.sort()

vcentral_relevancia = prob_relev[int(len(prob_relev)/2)]
vcentral_irrelevancia =  prob_nrelev[int(len(prob_relev)/2)]



for i in range(len(prob_relev)):
    if list_relevante[i]>=prob_relev[int(len(prob_relev)*0.75)]:
        muito_relevante+=1
    else:
        if list_relevante[i]>=prob_rel[int(len(prob_relev)*0.25)]:
            relevante+=1
        else:
            neutro+=1

for i in range(len(prob_nrelev)):
    if list_nrelevante[i]>=prob_nrelev[int(len(prob_nrelev)*0.75)]:
        muito_irrelevante+=1
    else:
        if list_nrelevante[i]>=prob_nrelev[int(len(prob_nrelev)*0.25)]:
            irrelevante+=1
        else:
            neutro+=1
            
p_false_posit=false_posit/df2.Teste.shape[0]
p_verd_posit=verd_posit/df2.Teste.shape[0]
p_false_negat=false_negat/df2.Teste.shape[0]
p_verd_negat=verd_negat/df2.Teste.shape[0]

print("Positivos verdadeiros:{0:.2f}%".format(p_verd_posit*100))
print("Negativos verdadeiros: {0:.2f}%".format(p_verd_negat*100))
print("Positivos falsos: {0:.2f}%".format(p_false_posit*100))
print("Negativos falsos: {0:.2f}%".format(p_false_negat*100))
print()
print()

print("Muito relevantes: {0:.2f}%".format((muito_relevante/len(df2.Relevância))*100))
print("Relevantes: {0:.2f}%".format((relevante/len(df2.Relevância))*100))
print("Neutros: {0:.2f}%".format((neutro/len(df2.Relevância))*100))
print("Irrelevantes: {0:.2f}%".format((irrelevante/len(df2.Relevância))*100))   
print("Muito irrelevantes: {0:.2f}%".format((muito_irrelevante/len(df2.Relevância))*100))

Positivos verdadeiros:56.00%
Negativos verdadeiros: 0.00%
Positivos falsos: 0.00%
Negativos falsos: 44.00%


Muito relevantes: 56.00%
Relevantes: 0.00%
Neutros: 0.00%
Irrelevantes: 0.00%
Muito irrelevantes: 44.00%


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


    Ao final do projeto, é possível dizer que o classificador é funcional, mas não da forma que se espera. O principal problema apresentado foi o fato dele apresentar apenas verdadeiros positivos/falsos negativos e muito relevantes/muito irrelevantes. Algumas explicações podem ser imaginadas para tal resultado: a primeira possibilidade seria ao realizar a classificação das bases no Excel, o que gerou uma quantidade muito grande de irrelevantes, o que tornaria a filtragem da base posteriormente mais grosseira, separando de forma simplória os dados, resultando nesses dois viés; outra explicação seria durante a programação do classificador, que falharia em realizar as operações de Naive Bayes e traria resultados ineficientes.
    As mensagens de dupla negação ou sarcasmo, ao passarem pelo classificador, passariam pelas duas possibilidades, negativo verdadeiro/positivo falso, resultando no direcionamento para tais regiões ou mesmo para os neutros. Para evitar que estes influenciassem os dados "reais", por assim dizer.
    O projeto cumpre, em partes, com seu objetivo, e deveria ser financiado, pois talvez o problema esteja nos dados gerados pelo produto em si, e não pelo desenvolvimento do classificador como um todo, ou seja, ao aplicar o classificador para o produto em questão do financiador, os dados gerados poderão ser mais conclusivos do que os dados gerados com base no produto utilizado. 
    Não é possível alimentar a base com novos tweets, pois ela usa a base gerada com a classificação prévia, ou seja, novos tweets não teriam uma pré-classificação para que o restante da ferramenta funciona de forma adequada.
    O classificador pode ser utilizado em outros cenários, como um simples classificador de emails do tipo spam, dizendo se o contéudo dentro deste se enquadra ou não na categoria; dizer se um trecho de texto expressa emoções positivas, como amor, felicidade, ou negativas, como raiva e tristeza; ou mesmo determinar o tipo categórico de um texto x dado para o classificador.
    Algumas melhorias podem ser feitas no classificador para que ele trabalhe melhor, entre elas, podemos citar: trabalhar com dados inexistentes, ou seja, adicionar dados que o próprio classificador não gera para melhorar as avaliações futuras; trabalhar com valores logaritmicos, visto que Naive Bayes trabalhar com valores muitos pequenos e isso gerará valores menores ainda, influenciando no resultado; ou usar outros métodos de distribuição que não a distribuição normal (ou gaussiana), podendo usar por exemplo a distribuição exponencial, que pode descrever melhor a distribuição de dados do classificador. 